# Advent of Code

## 2022-012-019
## 2022 019

https://adventofcode.com/2022/day/19

In [1]:
from collections import deque, defaultdict
from math import ceil
import re

# Parse the input file
def parse_input(file_path):
    blueprints = []
    with open(file_path, 'r') as f:
        for line in f:
            match = re.findall(r"\\d+", line)
            if match:
                blueprint_id = int(match[0])
                costs = list(map(int, match[1:]))
                blueprints.append((blueprint_id, {
                    'ore': {'ore': costs[0]},
                    'clay': {'ore': costs[1]},
                    'obsidian': {'ore': costs[2], 'clay': costs[3]},
                    'geode': {'ore': costs[4], 'obsidian': costs[5]}
                }))
    return blueprints

# Simulate the maximum geodes cracked for a given blueprint
def simulate(blueprint, time_limit):
    costs = blueprint[1]
    
    # State: (time, ore, clay, obsidian, geodes, ore_robots, clay_robots, obsidian_robots, geode_robots)
    initial_state = (0, 0, 0, 0, 0, 1, 0, 0, 0)
    
    queue = deque([initial_state])
    max_geodes = 0
    seen = set()

    while queue:
        state = queue.popleft()
        if state in seen:
            continue
        seen.add(state)
        time, ore, clay, obsidian, geodes, ore_robots, clay_robots, obsidian_robots, geode_robots = state

        # Update max geodes
        max_geodes = max(max_geodes, geodes)

        # Skip if out of time
        if time >= time_limit:
            continue

        # Generate new states by building robots or waiting
        for robot_type, cost in costs.items():
            # Check if we can afford the robot
            wait_time = 0
            for resource, amount in cost.items():
                if resource == 'ore':
                    wait_time = max(wait_time, ceil((amount - ore) / ore_robots) if ore_robots else float('inf'))
                elif resource == 'clay':
                    wait_time = max(wait_time, ceil((amount - clay) / clay_robots) if clay_robots else float('inf'))
                elif resource == 'obsidian':
                    wait_time = max(wait_time, ceil((amount - obsidian) / obsidian_robots) if obsidian_robots else float('inf'))

            # Check if it's possible to wait and build this robot within the time limit
            if time + wait_time + 1 > time_limit:
                continue

            # Calculate new resources after wait time
            new_time = time + wait_time + 1
            new_ore = ore + ore_robots * (wait_time + 1) - cost.get('ore', 0)
            new_clay = clay + clay_robots * (wait_time + 1) - cost.get('clay', 0)
            new_obsidian = obsidian + obsidian_robots * (wait_time + 1) - cost.get('obsidian', 0)
            new_geodes = geodes + geode_robots * (wait_time + 1)

            # Add new robot
            if robot_type == 'ore':
                new_state = (new_time, new_ore, new_clay, new_obsidian, new_geodes, ore_robots + 1, clay_robots, obsidian_robots, geode_robots)
            elif robot_type == 'clay':
                new_state = (new_time, new_ore, new_clay, new_obsidian, new_geodes, ore_robots, clay_robots + 1, obsidian_robots, geode_robots)
            elif robot_type == 'obsidian':
                new_state = (new_time, new_ore, new_clay, new_obsidian, new_geodes, ore_robots, clay_robots, obsidian_robots + 1, geode_robots)
            elif robot_type == 'geode':
                new_state = (new_time, new_ore, new_clay, new_obsidian, new_geodes, ore_robots, clay_robots, obsidian_robots, geode_robots + 1)

            queue.append(new_state)

        # Also consider just waiting until the end
        if geode_robots > 0:
            remaining_time = time_limit - time
            new_state = (
                time_limit,
                ore + ore_robots * remaining_time,
                clay + clay_robots * remaining_time,
                obsidian + obsidian_robots * remaining_time,
                geodes + geode_robots * remaining_time,
                ore_robots, clay_robots, obsidian_robots, geode_robots
            )
            queue.append(new_state)

    return max_geodes

# Calculate the total quality level
def calculate_quality_level(blueprints, time_limit=24):
    total_quality = 0
    for blueprint in blueprints:
        max_geodes = simulate(blueprint, time_limit)
        quality = blueprint[0] * max_geodes
        total_quality += quality
    return total_quality

# Main execution
file_path = 'input.txt'
blueprints = parse_input(file_path)
result = calculate_quality_level(blueprints)
print("Total Quality Level:", result)

Total Quality Level: 0
